In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
import streamlit as st
from langchain_groq import ChatGroq

from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader('oxford-mtrain-sow.pdf')
documents = loader.load()

In [5]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docText = textSplitter.split_documents(documents)

In [6]:
embeddings=OpenAIEmbeddings()
vector=FAISS.from_documents(docText, embeddings)

/home/kamlesh/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}
"""
)

In [8]:
llm=ChatGroq(model="llama3-8b-8192")

In [9]:
docChain = create_stuff_documents_chain(llm, prompt)
retriver = vector.as_retriever()
retriverChain = create_retrieval_chain(retriver, docChain)

In [10]:
query = "give me structure which is followed by the document. this is a SOW for a project. i need to follow this structure for my other projects."

response = retriverChain.invoke({"input": query})

In [11]:
response['answer']

'Based on the provided context, the document appears to be a Software Requirements Specification (SRS) document for a project. The structure followed in this document is:\n\n1. Date\n2. Business\n3. Requirements\n4. Specifications\n5. [Section name]\n\t* [Section content]\n6. [Section name]\n\t* [Section content]\n7. ...\n\nThe sections and their contents are:\n\n1. Date: The current date (24/12/2023)\n2. Business: A brief introduction to the project and its objectives\n3. Requirements: Defines the features of the Oxford Mtrain Web App development\n4. Specifications: Describes the requirements and specifications of the project\n5. Introduction: Provides an overview of the project, its objectives, and its primary objectives\n6. ... (additional sections may be present in the actual document)\n\nFor your other projects, you can follow a similar structure by including the following sections:\n\n1. Date\n2. Business\n3. Requirements\n4. Specifications\n5. Introduction\n6. [Section name] (e.